# ALGORITHME GENETIQUE POUR L'OPTIMISATION DE RESEAUX DE TRANSPORT

L'on implemente ici un Algorithme heuristique base sur l'algorithme genetique et l'algorithme SHAL pour la construction d'un reseaux de Arcs-hub dans un environnement competititif de Stackelberg entre deux entreprises de constructions de Reseaux de transport.
L'algorithme SHAL est utilise pour la construction de la fonction fitness utiliser par l'algo genetique

In [3]:
import random
from dataclasses import dataclass
from typing import Tuple
import numpy as np
import itertools
from time import time

In [4]:
class Node :
    pass

In [5]:

@dataclass(frozen=True)
class InstanceProbleme:
    noeuds: Tuple[Node, ...]
    matrice_demande_Wij: np.ndarray
    matrice_revenu_Fij: np.ndarray
    matrice_distance_dij: np.ndarray
    matrice_proportion_pij : np.ndarray
    facteur_rabais_alpha: float
    cout_collecte_chi: float
    cout_distribution_delta: float
    
    def __post_init__(self):
        # Vérification dimensions
        n = len(self.noeuds)

        for mat, name in [
            (self.matrice_demande_Wij, "demande"),
            (self.matrice_revenu_Fij, "revenu"),
            (self.matrice_distance_dij, "distance"),
            (self.matrice_proportion_pij, "proportion")
        ]:
            if mat.shape != (n, n):
                raise ValueError(f"Matrice {name} doit être de dimension ({n}, {n})")

        # Vérification paramètres scalaires
        if not (0 < self.facteur_rabais_alpha <= 1):
            raise ValueError("Le facteur de rabais alpha doit être dans ]0,1]")
        
        if self.cout_collecte_chi < 0 or self.cout_distribution_delta < 0:
            raise ValueError("Les coûts unitaires doivent être positifs")

        # Rendre les matrices non modifiables (immutabilité forte)
        object.__getattribute__(self, "matrice_demande_Wij").setflags(write=False)
        object.__getattribute__(self, "matrice_revenu_Fij").setflags(write=False)
        object.__getattribute__(self, "matrice_distance_dij").setflags(write=False)
        object.__getattribute__(self, "matrice_proportion_pij").setflags(write=False)

In [6]:
#Classe reprensentant les noeuds de notre future reseau de transport
class Node :
    def __init__(self, Id, name):
        self.Id = Id
        self.name = name
    def __repr__(self):
        return f"Node({self.Id}, '{self.name}')"
    def __eq__(self, other):
        if not isinstance(other, Node):
            return False
        return self.Id == other.Id and self.Id == other.Id
    def __hash__(self):
        return hash((self.Id))

In [7]:
# Classe reprensentant entierement la topologie d'un reseau c-a-d ses hubs et arcs-hubs 
# Possibilite de rajouter un dictionnaire renseignant tous les chemins optimaux deja calcules
# Un methode verifiant l'egalite entre 2 instances NetHub deja creer afin que tous deux partages leurs informations
# Mettre a jour les methodes en consequences
class NetHub :
    def __init__(self,
                 arcs_hub, # des couples d'instance Node definissant des arc-hubs
                 InstanceProbleme # Une InstanceProbleme definissant les conditions de transport(distance cout etc) 
                ):
        self.instance = InstanceProbleme
        self.arcs_hub = frozenset(arcs_hub)
        hub = []
        for i in arcs_hub :
            for j in range(len(i)):
                hub.append(i[j])
        self.hub = frozenset(hub)

    oldnet = {}

    def __repr__(self):
        """Affiche les IDs des nœuds hub"""
        for j in self.arcs_hub :
            print(j)
        return "[" + ", ".join(str(node.name) for node in self.hub) + "]"
        
    def __eq__(self, other):
        if not isinstance(other, NetHub):
            return False
            
        return ((frozenset(self.arcs_hub) == frozenset(other.arcs_hub)) and 
                (frozenset(self.hub) == frozenset(other.hub)))

    def __hash__(self):
        return hash((frozenset(self.arcs_hub),frozenset(self.hub)))

    def calculer_chemin_min_cout(self,o,d):
        """Calcule le chemin minimisant les couts entre 2 noeuds du probleme
        
        params :
                o : noeuds origine du chemin dans l' InstanceProbleme
                d : noeuds origine du chemin dans l' InstanceProbleme"""
        od_key = frozenset((o,d))
        if self.arcs_hub not in NetHub.oldnet :
            NetHub.oldnet[self.arcs_hub] = {}
        if od_key not in NetHub.oldnet[self.arcs_hub]:
            ip = self.instance
            O,D = (o.Id,
                   d.Id)
            ind_hub = sorted([i.Id for i in self.hub]) # Evite certains re-calcul
            best = np.inf
            path = []
            for k in ind_hub :
                for l in [i for i in ind_hub if i>=k]:
                    trial = (ip.cout_collecte_chi*ip.matrice_distance_dij[O,k] + 
                             ip.facteur_rabais_alpha*ip.matrice_distance_dij[k,l] + 
                             ip.cout_distribution_delta*ip.matrice_distance_dij[l,D] ) # On essaye un chemin
                    if trial < best : #si il a un cout moindre que le meilleur chemin so far 
                        best = trial #on le recupere comme meilleur chemin
                        path = [O,k,l,D] # Une liste d'Instance
                        dist = ( ip.matrice_distance_dij[O,k]+ 
                                ip.matrice_distance_dij[k,l] + 
                                ip.matrice_distance_dij[l,D] )
            
            
            NetHub.oldnet[self.arcs_hub][od_key] = (best, dist, path)
        
        return NetHub.oldnet[self.arcs_hub][od_key]
        
    @classmethod
    def reset(cls):
        cls.oldnet = {}
    

In [8]:
#Classe CustomerAllocation qui definit entierement comment se fait la repartition des clients
# Possibilite d'implementer un dictionnaire qui retient les repartitions deja faites afin d'eviter de reefectuer des calculs
# Ajout d'un attribut InstanceProbleme
class CustomerAllocation:
    def __init__(self,r1,r2):
        self.Customer_upper_threshold = max(r2,r1)
        self.Customer_lower_threshold = min(r1,r2)
        self.memorydic = {} 
        
    def allouer_demande(self, o, d, NetA, NetB):
        """Calcule comment les entreprises se repartiront les demandes sur un chemin
        
        params :
                o : noeuds dans l' InstanceProbleme origine du chemin
                d : noeuds dans l' InstanceProbleme destination du chemin
                NetA : Instance de NetHub
                NetB : Instance de NetHub
                """
        od_key = frozenset((o,d))
        if (NetA.arcs_hub,NetB.arcs_hub) not in self.memorydic:
            
            self.memorydic[(NetA.arcs_hub,NetB.arcs_hub)] = {}
            
            
        if od_key not in self.memorydic[(NetA.arcs_hub,NetB.arcs_hub)]:
            Ca,Da,_ = NetA.calculer_chemin_min_cout(o,d) # Calcule le chemin de A
            Cb,Db,_ = NetB.calculer_chemin_min_cout(o,d) # Calcule le chemin de B
            Cr = (Ca-Cb)/(Ca+Cb+1e-6) # Calcule les ratios 
            Dr = (Da-Db)/(Da+Db+1e-6)
            k = {"confort":[] ,"presse" : []}
            k_hub = {"confort":[] ,"presse" : []}
            if ( Cr <= -self.Customer_upper_threshold) :
                k["confort"].append((1.0,0.0))
                k_hub["confort"].append((1.0,0.0))
                
            elif (-self.Customer_upper_threshold < Cr <= -self.Customer_lower_threshold) :
                k_hub["confort"].append((1.0,0.0))
                k["confort"].append((0.75,0.25))
                    
            elif (-self.Customer_lower_threshold <Cr <= self.Customer_lower_threshold) :
                k_hub["confort"].append((0.75,0.25))
                k["confort"].append((0.50,0.50))
    
            elif (self.Customer_lower_threshold < Cr <= self.Customer_upper_threshold) :
                k_hub["confort"].append((0.75,0.25))
                k["confort"].append((0.25,0.75))
                
            elif (self.Customer_upper_threshold < Cr <= 1) :
                k_hub["confort"].append((0.50,0.50))
                k["confort"].append((0.0,1.0))
                
            if ( Dr <= -self.Customer_upper_threshold) :
                k_hub["presse"].append((1.0,0.0))
                k["presse"].append((1.0,0.0))
                
            elif (-self.Customer_upper_threshold < Dr <= -self.Customer_lower_threshold) :
                k_hub["presse"].append((1.0,0.0))
                k["presse"].append((0.75,0.25))
    
            elif (-self.Customer_lower_threshold < Dr <= self.Customer_lower_threshold) :
                k_hub["presse"].append((0.75,0.25))
                k["presse"].append((0.50,0.50))
    
            elif (self.Customer_lower_threshold < Dr <= self.Customer_upper_threshold) :
                k_hub["presse"].append((0.75,0.25))
                k["presse"].append((0.25,0.75))
                
            elif (self.Customer_upper_threshold < Dr <= 1) :
                k_hub["presse"].append((0.50,0.50))
                k["presse"].append((0.0,1.0))
                
            if o in NetA.hub :  # Penser a creer des getters (getHub sur NetHub)
                self.memorydic[(NetA.arcs_hub,NetB.arcs_hub)][od_key] =  k_hub
            else :
                self.memorydic[(NetA.arcs_hub,NetB.arcs_hub)][od_key] =  k
                
        return self.memorydic[(NetA.arcs_hub,NetB.arcs_hub)][od_key]

    def reset(self):
        self.memorydic = {}

In [38]:
#Classe Playground  centrale ou tout prend forme
# Creation d'un dictionnaire qui va recuperer les resultats d'un fitness un reseau leader (individu) 
#deja rencontrer
class Playground:
    def __init__(self, instance, nb_arcs_leader, nb_arcs_follower, custom_alloc,select ):
        self.instance = instance
        self.nb_arcs_leader = nb_arcs_leader
        self.nb_arcs_follower = nb_arcs_follower
        self.alloc = custom_alloc
        self.select = select
        self.memoryGen = {i : np.sum(instance.matrice_demande_Wij[i,:] * instance.matrice_revenu_Fij[i, :]) for i in range(len(instance.noeuds)) }
        self.memoryHubGeneration = {}
        self.memoryFit = {}
    
    
    def fitness(self, leader_net ):
        """Trouve la meilleure réponse du follower pour un réseau leader donné
            param:
                leader_net : une instance de NetHub
                k : lancer une approximation par Mc de la meilleur reponse avec sample """
        # Hubs du leader à exclure
        
        start1 = time()
        if not leader_net.arcs_hub in self.memoryFit :
            
            start1 = time()
            if self.select >0:
           
                follower_candidates = self._generate_hub_networks(
                    self.nb_arcs_follower,leader_net
                
                    )
                follower_candidates = follower_candidates[:len(follower_candidates)//self.select]
            else:
                follower_candidates = self._generate_hub_networks(
                    self.nb_arcs_follower,leader_net)
                
            
            
            best_net = None
            
            best_rev = -np.inf
            start = time()
            i = 0
            for follower_net in follower_candidates:
                
                # Calculer le revenu du follower
                
                
                leader_rev, follower_rev = self._revenue(leader_net, follower_net)
                
                
                if follower_rev >= best_rev:
                    best_rev = follower_rev
                    best_net = follower_net
                    score = leader_rev
            
            self.memoryFit[leader_net.arcs_hub]  = (best_net, score)
        
        return self.memoryFit[leader_net.arcs_hub]
    
    def _revenue(self, netA, netB):
        """Calcule les revenus des deux entreprises"""
        ip = self.instance
        n = len(ip.noeuds)
        revA = 0.0
        revB = 0.0
        
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                
                o = ip.noeuds[i]
                d = ip.noeuds[j]
                
                # Allocation de la demande
                Fract = self.alloc.allouer_demande(o, d, netA, netB)
                fracAp, fracBp = Fract["presse"][0]
                fracAc, fracBc = Fract["confort"][0]
                
                # Revenu pour cette paire OD
                demande = ip.matrice_demande_Wij[i, j]
                revenu = ip.matrice_revenu_Fij[i, j]
                prop = ip.matrice_proportion_pij[i, j]
                
                revA += (demande *prop* revenu * fracAp) + (demande *(1-prop)* revenu * fracAc)
                revB += (demande *prop* revenu * fracBp) +  (demande *(1-prop)* revenu * fracBc)
        
        return revA, revB
        
    
    def _generate_hub_networks(self, nb_arcs, leader_net, similarity_threshold=0.8):
        """
        Génère tous les réseaux de hubs possibles pour le follower
        - trie les noeuds selon leur score d'importance
        - réduit le score par 2 si le noeud est déjà dans le leader
        - inclut mémoire pour éviter recomputation
        - réutilise les réseaux précédents si un leader similaire a déjà été traité
        """
        ip = self.instance
        n = len(ip.noeuds)
        node_scores = self.memoryGen
        # Default placeholder node scores
        if node_scores is None:
            node_scores = {node: 1.0 for node in ip.noeuds}

        # Convert current leader hubs to set for similarity check
        current_hubs = set(leader_net.hub)

        if frozenset(current_hubs) in self.memoryHubGeneration :
            return self.memoryHubGeneration[frozenset(current_hubs)]

        else:
            # --- 2. Adjust node scores for leader hubs ---
            adjusted_scores = {}
            for node in ip.noeuds:
                score = node_scores.get(node, 1.0)
                if node in current_hubs:
                    score /= 2  # penalize nodes already in leader
                adjusted_scores[node] = score
    
            # --- 3. Generate all possible arcs and compute arc scores ---
            all_arcs = []
            for i in range(n):
                for j in range(i + 1, n):
                    node_i = ip.noeuds[i]
                    node_j = ip.noeuds[j]
                    arc_score = adjusted_scores[node_i] + adjusted_scores[node_j]
                    all_arcs.append(((node_i, node_j), arc_score))
    
            # Sort arcs descending by score
            all_arcs.sort(key=lambda x: x[1], reverse=True)
            arcs_list = [arc for arc, _ in all_arcs]
    
            # --- 4. Generate all combinations of nb_arcs arcs ---
            networks = []
            for combo in itertools.combinations(arcs_list, nb_arcs):
                network = NetHub(list(combo), ip)
                networks.append(network)
    
            # --- 5. Store in memory for future similar leaders ---
            self.memoryHubGeneration[frozenset(current_hubs)] = networks
    
            return networks
        
    def reset(self):
        self.memoryFit = {}
        self.memoryGen = {}
        self.memoryHubGeneration  = {}
    
    

## Outils Genetique

In [11]:
def random_individual(nodes,qA):
    arcs = []
    while len(arcs) < qA:
        i, j = random.sample(nodes, 2)
        if i != j and (i, j) not in arcs and (j, i) not in arcs:
            arcs.append(tuple([i, j]))
    return arcs


In [12]:
def tournament_selection(population,fitness,instance, k=3):
    contenders = random.sample(population, k)
    best_net = 0
    best_fit = 0
    for i in contenders :
        Net = NetHub(i,instance)
        fit = fitness(Net)[1]
        if fit > best_fit :
            best_net = i
            best_fit = fit
    return best_net,best_fit

In [13]:
def crossover(parent1, parent2,qA):
    cut = random.randint(1, qA - 1)
    child = parent1[:cut] + parent2[cut:]
    return list(set(child))[:qA]


In [14]:
def mutate(individual,nodes, rate=0.1):
    if random.random() < rate:
        idx = random.randint(0, len(individual)-1)
        i, j = random.sample(nodes, 2)
        individual[idx] = (i, j)
    return individual

In [15]:
def genetic_algorithm(
    instance,
    qA,
    repartition,
    select = 3,
    pop_size=10,
    generations=20,
    crossover_rate=0.8,
    mutation_rate=0.3
     
    
):
    ip = instance
    play = Playground(ip,qA,qA,repartition,select)
    fitness = play.fitness
    
    population = [random_individual(ip.noeuds,qA) for _ in range(pop_size)]
    best_solution = None
    best_score = -1

    for gen in range(generations):
        new_population = []
        current_best = 0
        current_score = -1
        for _ in range(pop_size):
            parent1,score = tournament_selection(population,fitness,ip)
            
            if current_score < score :
                current_score = score
                current_best  = parent1
            if random.random() < crossover_rate:
                parent2 = tournament_selection(population,fitness,ip)[0]
                child = crossover(parent1, parent2,qA)
                
            else:
                child = parent1.copy()

            child = mutate(child,ip.noeuds, rate =  mutation_rate)
            new_population.append(child)

        population = new_population

        

        if current_score > best_score:
            best_score = current_score
            best_solution = current_best

        print(f"Génération {gen} | Meilleure fitness = {best_score}")
    NetHub.reset()
    repartition.reset()
    play.reset()
    return best_solution, best_score

##

In [17]:
import numpy as np

# --- Nœuds ---
noeuds = tuple(
    Node(i, nom) for i, nom in enumerate([
        'N1', 'N2', 'N3', 'N4', 'N5'
    ])
)

# --- Coordonnées internes ---
coords = {
    'N1':  (0.2, 1.1),
    'N2':  (-0.5, 0.3),
    'N3':  (1.0, -0.4),
    'N4':  (-1.2, -0.8),
    'N5':  (0.6, -1.0),

}

n = len(noeuds)

# --- Distance euclidienne ---
distance = np.zeros((n, n))
for i in range(n):
    xi, yi = coords[noeuds[i].name]
    for j in range(n):
        if i != j:
            xj, yj = coords[noeuds[j].name]
            distance[i, j] = np.sqrt((xi - xj)**2 + (yi - yj)**2)

distance = np.round(distance, 3)

# --- Revenu cohérent avec la distance ---
revenu = np.zeros((n, n))
mask = distance > 0
revenu[mask] = np.round(1 / distance[mask], 3)

# --- Demande entière ---
rng = np.random.default_rng(42)
demande = rng.integers(1, 100, size=(n, n))
demande = np.triu(demande, 1)
demande = demande + demande.T
np.fill_diagonal(demande, 0)

# --- Proportion ---
proportion = rng.random((n, n))
proportion = np.triu(proportion, 1)
proportion = proportion + proportion.T
np.fill_diagonal(proportion, 0)
proportion = np.round(proportion, 3)

# --- Instance du problème ---
instance_data = InstanceProbleme(
    noeuds=noeuds,
    matrice_demande_Wij=demande,
    matrice_revenu_Fij=revenu,
    matrice_distance_dij=distance,
    matrice_proportion_pij=proportion,
    facteur_rabais_alpha=0.8,
    cout_collecte_chi=1.0,
    cout_distribution_delta=1.0
)

In [18]:
instance_data.matrice_demande_Wij

array([[ 0, 77, 65, 44, 43],
       [77,  0, 70, 20, 10],
       [65, 70,  0, 76, 72],
       [44, 20, 76,  0, 45],
       [43, 10, 72, 45,  0]], dtype=int64)

In [19]:
instance_data.matrice_proportion_pij

array([[0.   , 0.443, 0.227, 0.555, 0.064],
       [0.443, 0.   , 0.758, 0.355, 0.971],
       [0.227, 0.758, 0.   , 0.467, 0.044],
       [0.555, 0.355, 0.467, 0.   , 0.326],
       [0.064, 0.971, 0.044, 0.326, 0.   ]])

In [20]:
Test = CustomerAllocation(0.6,0.4)

In [21]:
nodes = [Node(i, f"Ville_{i}") for i in range(5)]  # Augmenté à 5 nœuds
    
np.random.seed(42)
n = len(nodes)
W = np.random.rand(n, n) * 100
F = np.random.rand(n, n) * 10
D = np.random.rand(n, n) * 100
P = np.random.random((n,n))
W = W + W.T
F = F + F.T
D = D + D.T
P = (P + P.T)*0.5
for i in range(n):
    W[i,i] = 0
    F[i,i] = 0
    D[i,i] = 0
    P[i,i] = 0
    # Créer l'instance
instance = InstanceProbleme(
        noeuds=tuple(nodes),
        matrice_demande_Wij=W,
        matrice_revenu_Fij=F,
        matrice_distance_dij=D,
        matrice_proportion_pij=P, 
        facteur_rabais_alpha=0.6,
        cout_collecte_chi=1.0,
        cout_distribution_delta=1.0
    )


In [22]:
start = time()
#best_arcs, best_value = genetic_algorithm(instance,2,Test,20,pop_size= 70,generations=3,crossover_rate=0.8,mutation_rate= 0.2)

#print("\nMeilleure solution trouvée :")
#print("Arcs hubs du leader :", best_arcs)
#print("Revenu capturé :", best_value)
#
#print(f"Duree du processus :{time() - start}")

In [23]:
play = Playground(instance,2,2,Test,1)
fitness = play.fitness

# DONNEE

In [25]:
import pandas as pd

In [26]:
k = 0

In [27]:
url = f"Donnee{k}.xlsx"

In [28]:
print(url)

Donnee0.xlsx


In [29]:
Data1 = pd.read_excel("Données1.xlsx",sheet_name = None)

In [30]:
nodes = [Node(i, f"Ville_{i}") for i in range(1,16)]
Matrices = {}
for key in Data1 :
    df = Data1[key]
    mat = df.drop([df.columns[0]], axis = 1)
    mat = mat.to_numpy()
    Matrices[key] = mat

In [31]:
def Experiment(i: int,
               cust_selectivity,
               arcs_hub = 2,
               fitness_exploration = 20,
               facteur_rabais_alpha1=0.8,
               cout_collecte_chi=1.0,
               cout_distribution_delta=1.0,
               pop_size = 90,
               generations = 15
              ):
    path = f"Données{i}.xlsx"
    Allocation = CustomerAllocation(cust_selectivity[0],cust_selectivity[1])
    nodes = [Node(i, f"Ville_{i}") for i in range(15)]
    Data1 = pd.read_excel(path,sheet_name = None)
    Matrixes = {}
    for key in Data1 :
        df = Data1[key]
        mat = df.drop([df.columns[0]], axis = 1)
        mat = mat.to_numpy()
        Matrixes[key] = mat
    instance = InstanceProbleme(
        noeuds=tuple(nodes),
        matrice_demande_Wij=Matrixes["Demande"],
        matrice_revenu_Fij=Matrixes["Revenus"],
        matrice_distance_dij=Matrixes["Distances"],
        matrice_proportion_pij=Matrixes["Proportion"], 
        facteur_rabais_alpha=facteur_rabais_alpha1,
        cout_collecte_chi=1.0,
        cout_distribution_delta=1.0
    )
    print("Environnement d'experimentation :")
    print("\nDataset : "+path  +f"\nSeuils de selectivite des clients : {cust_selectivity}\nEconomie d'echelle sur les arcs_hub :{facteur_rabais_alpha1}\nArcs_hub par firme :{arcs_hub}" )
    print(f"\nLancement de l'algorithme Genetique:(Exploration de fitness :{1/fitness_exploration})")
    start = time()
    best_arcs, best_value = genetic_algorithm(instance,
                                              arcs_hub,
                                              Allocation,
                                              fitness_exploration,
                                              pop_size= pop_size,
                                              generations= generations,
                                              crossover_rate= 0.8,
                                              mutation_rate= 0.2)
    
    print("\nMeilleure solution trouvée :")
    print("Arcs hubs du leader :", best_arcs)
    print("Revenu capturé :", best_value)
    
    print(f"Duree du processus :{time() - start}")
    
    play = Playground(instance,arcs_hub,arcs_hub,Allocation,1)
    fit = play.fitness
    print("\nCalcul de la meilleur reponse du follower pour la solution.... ")
    sol_b, revenue = fit(NetHub(best_arcs,instance))
    print("Arcs hubs follower : ",sol_b)
    print("Revenu reel capturé par la solution :", revenue)
    whole_cake = np.sum(Matrixes["Demande"]*Matrixes["Revenus"])
    print("Revenu total sur le marche :", whole_cake)

In [32]:
instance = InstanceProbleme(
        noeuds=tuple(nodes),
        matrice_demande_Wij=Matrices["Demande"],
        matrice_revenu_Fij=Matrices["Revenus"],
        matrice_distance_dij=Matrices["Distances"],
        matrice_proportion_pij=Matrices["Proportion"], 
        facteur_rabais_alpha=0.5,
        cout_collecte_chi=1.0,
        cout_distribution_delta=1.0
    )
    

In [40]:
for i in range(1,9):
    print("Debut de l'experience ",i)
    print("="*30)
    Experiment(i,(0.5,0.1),facteur_rabais_alpha1=0.5)
    print("="*25)

Debut de l'experience  1
Environnement d'experimentation :

Dataset : Données1.xlsx
Seuils de selectivite des clients : (0.5, 0.1)
Economie d'echelle sur les arcs_hub :0.5
Arcs_hub par firme :2

Lancement de l'algorithme Genetique:(Exploration de fitness :0.05)
Génération 0 | Meilleure fitness = 437900301590.62445
Génération 1 | Meilleure fitness = 440990847656.0648
Génération 2 | Meilleure fitness = 448731840774.7382
Génération 3 | Meilleure fitness = 448731840774.7382
Génération 4 | Meilleure fitness = 448731840774.7382
Génération 5 | Meilleure fitness = 448731840774.7382
Génération 6 | Meilleure fitness = 448731840774.7382
Génération 7 | Meilleure fitness = 448731840774.7382
Génération 8 | Meilleure fitness = 448731840774.7382
Génération 9 | Meilleure fitness = 448731840774.7382
Génération 10 | Meilleure fitness = 448731840774.7382
Génération 11 | Meilleure fitness = 448731840774.7382
Génération 12 | Meilleure fitness = 448731840774.7382
Génération 13 | Meilleure fitness = 448731840

In [ ]:
start = time()
best_arcs, best_value = genetic_algorithm(instance,2,Test,20,pop_size= 70,generations=5,crossover_rate=0.8,mutation_rate= 0.2)

print("\nMeilleure solution trouvée :")
print("Arcs hubs du leader :", best_arcs)
print("Revenu capturé :", best_value)

print(f"Duree du processus :{time() - start}")

In [ ]:
print(np.sum(Matrices["Demande"]*Matrices["Revenus"])- 7562170655.341133)